### Oppgave 3) Databehandling

Målet vårt med temperaturdataen er å se etter en trend som kan si noe om utviklingen til klimaet, for at dette skal være mulig trenger vi at all data stemmer, og ikke minst at det ikke mangler noe data

Vi vil først søke etter tomme verdier for å sikre at dataen vi har er komplett

In [3]:
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# API request parameters
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 63.43,
    "longitude": 10.39,
    "start_date": "1950-01-01",
    "end_date": "2025-03-22",
    "hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

# Create a DataFrame
hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature_2m
}


# Check for missing values
missing_values = hourly_dataframe["temperature_2m"].isna().sum()
print(f"Number of missing temperature values: {missing_values}")




Coordinates 63.444637298583984°N 10.36649227142334°E
Elevation 14.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
Number of missing temperature values: 0


Koden over søkte etter tomme verdier, og fant 0 tomme verdier. Videre vil vi nå søke etter urealistiske målinger. Vi har undesøkt og vet at det ikke kan bli kaldere enn -40 grader i Trondheim og heller ikke varmere enn 40 grader her, dermed skal vi søke etter målinger som enten er kaldere eller varmere enn gitte temperaturer ettersom det er målinger som ikke kan være realistiske

In [8]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 63.43,
    "longitude": 10.39,
    "start_date": "1950-01-01",
    "end_date": "2025-03-22",
    "hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature_2m
}

hourly_dataframe = pd.DataFrame(data=hourly_data)

# Feilsøking: Finn temperaturer utenfor realistisk område
min_temp = -40  # Laveste mulige realistiske temperatur på jorda
max_temp = 40    # Høyeste mulige realistiske temperatur på jorda

# Filtrer ut urealistiske temperaturverdier
unrealistic_temperatures = hourly_dataframe[
    (hourly_dataframe["temperature_2m"] < min_temp) | 
    (hourly_dataframe["temperature_2m"] > max_temp)
]

# Skriv ut urealistiske verdier hvis noen finnes
if not unrealistic_temperatures.empty:
    print("\n⚠️ Urealistiske temperaturverdier funnet:")
    print(unrealistic_temperatures)
else:
    print("\n✅ Ingen urealistiske temperaturer funnet.")





Coordinates 63.444637298583984°N 10.36649227142334°E
Elevation 14.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s

✅ Ingen urealistiske temperaturer funnet.


Koden fant heller ingen urealistiske data